# DeepMatch 样例代码
- https://github.com/shenweichen/DeepMatch
- https://deepmatch.readthedocs.io/en/latest/

# 导入需要的库

In [1]:
user=['age','gender' , 'residence' , 'city', 'city_rank', 'series dev', 'series group', 'emui dev','device name','device size', 'net_type']      
item=['task id', 'adv id','creat_type_cd','adv_prim id', 'inter type cd','slot id', 'spread app id','app second class','log_id']

In [2]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input,gen_data_set_sdm,gen_model_input_sdm
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler
import random
import numpy as np
from collections import defaultdict  
import os, math, warnings, math, pickle
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat,DenseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
from sklearn.metrics import log_loss, roc_auc_score
warnings.filterwarnings('ignore')
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

In [3]:
import pandas as pd
train_ads = pd.read_pickle('d:/Data/2022_3_data/train/train_data_ads.pkl')
train_ads["time_YMD"]=train_ads["pt_d"]//10000-20220603
train_ads["hour"]=train_ads["pt_d"]%10000//100
data=train_ads.copy()

# 读取数据

In [4]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','movie_id','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['movie_id','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000]

# 构建特征列，训练模型，导出embedding

In [5]:
train_ads.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle',
       'time_YMD', 'hour'],
      dtype='object')

In [6]:
def gen_data_set_sdm(data, seq_short_max_len=5, seq_prefer_max_len=50):
    data.sort_values("pt_d", inplace=True)
    item_ids = data['adv_id'].unique()
    item_id_genres_map = dict(zip(data['adv_id'].values, data['creat_type_cd'].values))
    train_set, test_set = [], []
    for user_id, hist_click in tqdm(data.groupby('user_id')):
        # 这里按照expo_date分开，每一天用滑动窗口滑，可能相关性更高些,另外，这样序列不会太长，因为eda发现有点击1111个的
        #for expo_date, hist_click in hist_date_click.groupby('expo_date'):
        # 用户当天的点击历史id
        pos_list = hist_click['adv_id'].tolist()
        label_list=hist_click['label'].tolist()
        pt_d_list=hist_click["pt_d"].tolist()
        age_list=hist_click["age"].tolist()
        gender_list=hist_click["gender"].tolist()
        residence_list=hist_click["residence"].tolist()
        city_list=hist_click["city"].tolist()
        city_rank_list=hist_click["city_rank"].tolist()
        series_dev_list=hist_click['series_dev'].tolist()
        series_group_list=hist_click['series_group'].tolist()
        emui_dev_list=hist_click['emui_dev'].tolist()
        device_name_list=hist_click['device_name'].tolist()
        device_size_list=hist_click['device_size'].tolist()
        net_type_list=hist_click['net_type'].tolist()
        task_id_list=hist_click["task_id"].tolist()
        create_type_list=hist_click["creat_type_cd"].tolist()
        adv_prim_list=hist_click["adv_prim_id"].tolist()
        inter_tyepe_list=hist_click["inter_type_cd"].tolist()
        slot_id_list=hist_click["slot_id"].tolist()
        spread_app_id_list=hist_click["spread_app_id"].tolist()
        app_second_id_list=hist_click["app_second_class"].tolist()

        #user_control_flag = True
        
        
        # 过长的序列截断
       
        #if negsample > 0:
        #    neg_list = gen_neg_sample_candiate(pos_list, item_ids, doc_clicked_count_dict, negsample, methods='multinomial')
        
        # 只有1个的也截断 去掉，当然我之前做了处理，这里没有这种情况了
        if len(pos_list) < 1:
            continue
        else: 
            for i in range(0, len(pos_list)):
                hist = pos_list[:i]
                genres_hist = create_type_list[:i]
                seq_short_len = min(i, seq_short_max_len)
                seq_prefer_len = min(max(i - seq_short_len, 0), seq_prefer_max_len)
                row = [user_id,  hist[::-1][:seq_short_len][::-1], pos_list[i],age_list[i],gender_list[i],residence_list[i],city_list[i]
                ,city_rank_list[i],series_dev_list[i],series_group_list[i],emui_dev_list[i],device_name_list[i], 
                device_size_list[i],net_type_list[i],label_list[i],len(hist[::-1]),task_id_list[i],
                create_type_list[i],adv_prim_list[i],inter_tyepe_list[i],slot_id_list[i],
                spread_app_id_list[i],app_second_id_list[i],
                hist[::-1][seq_short_len:seq_short_len + seq_prefer_len] ,seq_short_len,
                     seq_prefer_len, genres_hist[::-1][:seq_short_len][::-1],
                     genres_hist[::-1][seq_short_len:seq_short_len + seq_prefer_len],]
                if pt_d_list[i]<202206070000:
                    train_set.append(row)
                else:
                    test_set.append(row)
# hist[::-1][:seq_len], seq_len, genres_hist[::-1][:seq_len],
#                    genres_list[i],
#                    rating_list[i]))
              
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    return train_set, test_set   


In [7]:
# task_id_list=hist_click["task_id"].tolist()
#         create_type_list=hist_click["creat_type_cd"].tolist()
#         adv_prim_list=hist_click["adv_prim_id"].tolist()
#         inter_tyepe_list=hist_click["inter_type_cd"].tolist()
#         slot_id_list=hist_click["slot_id"].tolist()
#         spread_app_id_list=hist_click["spread_app_id"].tolist()
#         app_second_id_list=hist_click["app_second_id"].tolist()
#         log_id_list=hist_click["log_id"].tolist()

def gen_model_input_sdm(train_set, user_profile, seq_short_max_len, seq_prefer_max_len):
    train_uid = np.array([row[0] for row in train_set])
    short_train_seq = [row[1] for row in train_set]
    prefer_train_seq = [line[23] for line in train_set]
    train_iid = np.array([row[2] for row in train_set])
    train_u_age = np.array([row[3] for row in train_set])
    train_u_gender = np.array([row[4] for row in train_set])
    train_u_residence=np.array([row[5] for row in train_set])
    train_u_city = np.array([row[6] for row in train_set])
    train_u_city_rank = np.array([row[7] for row in train_set])
    train_u_series_dev=np.array([row[8] for row in train_set])
    train_u_series_group=np.array([row[9] for row in train_set])
    train_u_emui_dev=np.array([row[10] for row in train_set])
    train_u_device_name = np.array([row[11] for row in train_set])
    train_u_device_size = np.array([row[12] for row in train_set])
    train_u_net_type_list=np.array([row[13] for row in train_set])
    train_label = np.array([row[14] for row in train_set])
    train_hist_len = np.array([row[15] for row in train_set])
    
    train_task_id=np.array([row[16]for row in train_set])
    train_create_type=np.array([row[17] for row in train_set])
    train_adv_prim=np.array([row[18] for row in train_set])
    train_inter_tyepe_list=np.array([row[19] for row in train_set])
    train_slot_id_list=np.array([row[20] for row in train_set])
    train_spread_app_id_list=np.array([row[21] for row in train_set])
    train_app_second_id_list=np.array([row[22] for row in train_set])
    
    train_short_len = np.array([line[24] for line in train_set])
    train_prefer_len = np.array([line[25] for line in train_set])
    short_train_seq_genres = np.array([line[26] for line in train_set])
    prefer_train_seq_genres = np.array([line[27] for line in train_set])


    train_short_item_pad = pad_sequences(short_train_seq, maxlen=seq_short_max_len, padding='post', truncating='post',
                                         value=0)
    train_prefer_item_pad = pad_sequences(prefer_train_seq, maxlen=seq_prefer_max_len, padding='post',
                                          truncating='post',
                                          value=0)
    train_short_genres_pad = pad_sequences(short_train_seq_genres, maxlen=seq_short_max_len, padding='post',
                                           truncating='post',
                                           value=0)
    train_prefer_genres_pad = pad_sequences(prefer_train_seq_genres, maxlen=seq_prefer_max_len, padding='post',
                                            truncating='post',
                                            value=0)
    
    train_model_input = {
        "user_id": train_uid,
        "adv_id": train_iid,
        "short_movie_id": train_short_item_pad,
        "prefer_movie_id": train_prefer_item_pad,
        "hist_len": train_hist_len,
        "prefer_sess_length": train_prefer_len,
        "short_sess_length": train_short_len, 
        'short_genres': train_short_genres_pad,
        'prefer_genres': train_prefer_genres_pad,
        "u_city": train_u_city,
        "u_age": train_u_age,
        "u_gender": train_u_gender,
        "residence": train_u_residence,
        "city_rank":train_u_city_rank,
        "series_dev":train_u_series_dev,
        "series_group":train_u_series_group,
        "emui_dev":train_u_emui_dev,
        "device_name":train_u_device_name,
        "device_size":train_u_device_size,
        "net_type_list":train_u_net_type_list,
        
        "task_id":train_task_id,
        "create_type":train_create_type,
        "adv_prim":train_adv_prim,
        "inter_tyepe":train_inter_tyepe_list,
        "slot_id":train_slot_id_list,
        "spread_app_id":train_spread_app_id_list,
        "pp_second_id":train_app_second_id_list
    }

   

    # train_model_input = {"user_id": train_uid, "movie_id": train_iid, "short_movie_id": train_short_item_pad,
    #                      "prefer_movie_id": train_prefer_item_pad,
    #                      "prefer_sess_length": train_prefer_len,
    #                      "short_sess_length": train_short_len, 'short_genres': train_short_genres_pad,
    #                      'prefer_genres': train_prefer_genres_pad}

    # for key in ["gender", "age", "occupation", "zip"]:
    #     train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input, train_label

In [8]:
 #获取用户embedding和文章embedding
def get_embeddings(model, test_model_input, user_idx_2_rawid, doc_idx_2_rawid, save_path='d:/Data/2022_3_data/train/'):
    doc_model_input = {'adv_id':np.array(list(doc_idx_2_rawid.keys()))}
    
    user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
    doc_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)
    
    # 保存当前的item_embedding 和 user_embedding 排序的时候可能能够用到，但是需要注意保存的时候需要和原始的id对应
    user_embs = user_embedding_model.predict(test_model_input, batch_size=2 ** 12)
    doc_embs = doc_embedding_model.predict(doc_model_input, batch_size=2 ** 12)
    # embedding保存之前归一化一下
    user_embs = user_embs / np.linalg.norm(user_embs, axis=1, keepdims=True)
    doc_embs = doc_embs / np.linalg.norm(doc_embs, axis=1, keepdims=True)
    
    # 将Embedding转换成字典的形式方便查询
    raw_user_id_emb_dict = {user_idx_2_rawid[k]: \
                                v for k, v in zip(user_idx_2_rawid.keys(), user_embs)}
    raw_doc_id_emb_dict = {doc_idx_2_rawid[k]: \
                                v for k, v in zip(doc_idx_2_rawid.keys(), doc_embs)}
    # 将Embedding保存到本地
    pickle.dump(raw_user_id_emb_dict, open(save_path + 'user_youtube_emb.pkl', 'wb'))
    pickle.dump(raw_doc_id_emb_dict, open(save_path + 'doc_youtube_emb.pkl', 'wb'))
    
    # 读取
    #user_embs_dict = pickle.load(open('embedding/user_youtube_emb.pkl', 'rb'))
    #doc_embs_dict = pickle.load(open('embedding/doc_youtube_emb.pkl', 'rb'))
    return raw_user_id_emb_dict, raw_doc_id_emb_dict


In [9]:
train_ads.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle',
       'time_YMD', 'hour'],
      dtype='object')

In [10]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
sparse_features=['time_YMD','hour', 'task_id','adv_id', 'creat_type_cd', 'user_id','age', 'gender','residence','city_rank', 'series_dev','series_group','emui_dev','adv_prim_id','device_name','device_size','city','net_type','inter_type_cd','slot_id', 'spread_app_id', 'app_second_class' ]
SEQ_LEN = 50
SEQ_LEN_short = 5
SEQ_LEN_prefer = 50
negsample = 0

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
features = sparse_features
user_profile = data[features].drop_duplicates('user_id')

item_profile = data[["adv_id"]].drop_duplicates('adv_id')

user_profile.set_index("user_id", inplace=True)

user_item_list = data.groupby("user_id")['adv_id'].apply(list)

train_set, test_set = gen_data_set_sdm(data, seq_short_max_len=SEQ_LEN_short, seq_prefer_max_len=SEQ_LEN_prefer)

train_model_input, train_label = gen_model_input_sdm(train_set, user_profile, SEQ_LEN_short, SEQ_LEN_prefer)
test_model_input, test_label = gen_model_input_sdm(test_set, user_profile, SEQ_LEN_short, SEQ_LEN_prefer)

# 2.count #unique features for each sparse field and generate feature config for sequence feature

embedding_dim = 32
user_id_raw = data[['user_id']].drop_duplicates('user_id',keep="last")
doc_id_raw = data[['adv_id']].drop_duplicates('adv_id',keep="last")
user_id_enc = data[['user_id']].drop_duplicates('user_id',keep="last")
doc_id_enc = data[['adv_id']].drop_duplicates('adv_id',keep="last")
user_idx_2_rawid = dict(zip(user_id_enc['user_id'], user_id_raw['user_id']))
doc_idx_2_rawid = dict(zip(doc_id_enc['adv_id'], doc_id_raw['adv_id']))
user_feature_columns = [
    SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
    SparseFeat('u_city', feature_max_idx['city'], embedding_dim),
        SparseFeat('u_age', feature_max_idx['age'], embedding_dim),
        SparseFeat('u_gender', feature_max_idx['gender'], embedding_dim),
        SparseFeat('residence',feature_max_idx["residence"],embedding_dim),
        SparseFeat('city_rank',feature_max_idx["city_rank"],embedding_dim),
        SparseFeat('series_dev',feature_max_idx["series_dev"],embedding_dim),
        SparseFeat('series_group',feature_max_idx["series_group"],embedding_dim),
        SparseFeat('emui_dev',feature_max_idx["emui_dev"],embedding_dim),
        SparseFeat('device_name',feature_max_idx["device_name"],embedding_dim),
        SparseFeat('device_size',feature_max_idx["device_size"],embedding_dim),
        SparseFeat('net_type_list',feature_max_idx["net_type"],embedding_dim),
       # DenseFeat('hist_len', 1,),   
                        VarLenSparseFeat(SparseFeat('short_movie_id', feature_max_idx['adv_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN_short, 'mean',
                                         'short_sess_length'),
                        VarLenSparseFeat(SparseFeat('prefer_movie_id', feature_max_idx['adv_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN_prefer, 'mean',
                                         'prefer_sess_length'),
                        VarLenSparseFeat(SparseFeat('short_genres', feature_max_idx['creat_type_cd'], embedding_dim,
                                                    embedding_name="genres"), SEQ_LEN_short, 'mean',
                                         'short_sess_length'),
                        VarLenSparseFeat(SparseFeat('prefer_genres', feature_max_idx['creat_type_cd'], embedding_dim,
                                                    embedding_name="genres"), SEQ_LEN_prefer, 'mean',
                                         'prefer_sess_length'),]
#                         ]
#  "task_id":train_task_id,
#         "create_type":train_create_type,
#         "adv_prim":train_adv_prim,
#         "inter_tyepe":train_inter_tyepe_list,
#         "slot_id":train_slot_id_list,
#         "spread_app_id":train_spread_app_id_list,
#         "pp_second_id":train_app_second_id_list



item_feature_columns = [SparseFeat('adv_id', feature_max_idx['adv_id'], embedding_dim),
# SparseFeat('task_id', feature_max_idx['task_id'], embedding_dim),
# SparseFeat('adv_prim', feature_max_idx['adv_prim_id'], embedding_dim),
# SparseFeat('inter_tyepe', feature_max_idx['inter_type_cd'], embedding_dim),
# SparseFeat('slot_id', feature_max_idx['slot_id'], embedding_dim),
# SparseFeat('spread_app_id', feature_max_idx['spread_app_id'], embedding_dim),
# SparseFeat('pp_second_id', feature_max_idx['app_second_class'], embedding_dim),


]

from collections import Counter
train_counter = Counter(train_model_input['adv_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="adv_id",item_count=item_count)

# 3.Define Model and train

import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

model = SDM(user_feature_columns, item_feature_columns, history_feature_list=['movie_id','genres'],
            units=embedding_dim, sampler_config=sampler_config )

model.compile(optimizer="adam", loss=sampledsoftmaxloss)

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=64, epochs=20, verbose=1, validation_split=0.0, )
full_data=train_set+test_set
full_data_input,full_data_label=gen_model_input(full_data)
# 4. Generate user features for testing and full item features for retrieval
user_embs, doc_embs = get_embeddings(model, full_data_input, user_idx_2_rawid, doc_idx_2_rawid)

 20%|█▉        | 12751/65297 [00:21<01:27, 597.77it/s] 


KeyboardInterrupt: 

In [ ]:
user_feature_columns = [
    SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
    SparseFeat('u_city', feature_max_idx['city'], embedding_dim),
        SparseFeat('u_age', feature_max_idx['age'], embedding_dim),
        SparseFeat('u_gender', feature_max_idx['gender'], embedding_dim),
        SparseFeat('residence',feature_max_idx["residence"],embedding_dim),
        SparseFeat('city_rank',feature_max_idx["city_rank"],embedding_dim),
        SparseFeat('series_dev',feature_max_idx["series_dev"],embedding_dim),
        SparseFeat('series_group',feature_max_idx["series_group"],embedding_dim),
        SparseFeat('emui_dev',feature_max_idx["emui_dev"],embedding_dim),
        SparseFeat('device_name',feature_max_idx["device_name"],embedding_dim),
        SparseFeat('device_size',feature_max_idx["device_size"],embedding_dim),
        SparseFeat('net_type_list',feature_max_idx["net_type"],embedding_dim),
        #DenseFeat('hist_len', 1,),   
                        VarLenSparseFeat(SparseFeat('short_movie_id', feature_max_idx['adv_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN_short, 'mean',
                                         'short_sess_length'),
                        VarLenSparseFeat(SparseFeat('prefer_movie_id', feature_max_idx['adv_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN_prefer, 'mean',
                                         'prefer_sess_length'),
                        VarLenSparseFeat(SparseFeat('short_genres', feature_max_idx['creat_type_cd'], embedding_dim,
                                                    embedding_name="genres"), SEQ_LEN_short, 'mean',
                                         'short_sess_length'),
                        VarLenSparseFeat(SparseFeat('prefer_genres', feature_max_idx['creat_type_cd'], embedding_dim,
                                                    embedding_name="genres"), SEQ_LEN_prefer, 'mean',
                                         'prefer_sess_length')]

In [ ]:
item_feature_columns = [SparseFeat('adv_id', feature_max_idx['adv_id'], embedding_dim)
# SparseFeat('task_id', feature_max_idx['task_id'], embedding_dim),
# SparseFeat('adv_prim', feature_max_idx['adv_prim_id'], embedding_dim),
# SparseFeat('inter_tyepe', feature_max_idx['inter_type_cd'], embedding_dim),
# SparseFeat('slot_id', feature_max_idx['slot_id'], embedding_dim),
# SparseFeat('spread_app_id', feature_max_idx['spread_app_id'], embedding_dim),
# SparseFeat('pp_second_id', feature_max_idx['app_second_class'], embedding_dim),


]

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['adv_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="adv_id",item_count=item_count)

# 3.Define Model and train

import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

model = SDM(user_feature_columns, item_feature_columns, history_feature_list=['movie_id','genres'],
            units=embedding_dim, sampler_config=sampler_config )

model.compile(optimizer="adam", loss=sampledsoftmaxloss)

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=64, epochs=20, verbose=1, validation_split=0.0, )
full_data=train_set+test_set
full_data_input,full_data_label=gen_model_input(full_data)
# 4. Generate user features for testing and full item features for retrieval
user_embs, doc_embs = get_embeddings(model, full_data_input, user_idx_2_rawid, doc_idx_2_rawid)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
